In [1]:
!pip install simpy
import simpy, random, statistics, pandas as pd

LAMBDA = 5
mean_id = 0.75
scan_min = 0.5
scan_max = 1.0
WARMUP = 30  # ignore first half hour for stats
sim_time = 10080 # run for a week
random_seed = 42

random.seed(random_seed)

def exp_time(mean):
    return random.expovariate(1/mean)

# choose the resource with the fewest queued requests
def choose_shortest(resources):
    return min(resources, key=lambda r: len(r.queue))
    
#  Process for each passenger moving through the system.
def passenger (env, name, id_checkers, scanners, stats):
    
    arrival_time = env.now

    # ID/Boarding-pass check queue
    t_enter_id_q = env.now
    with id_checkers.request() as req:
        yield req
        t_start_id = env.now
        # queue wait at ID:
        wait_id = t_start_id - t_enter_id_q
        yield env.timeout(exp_time(mean_id)) # service
        
    # Personal Scanner (route to shortest queue)
    t_enter_scan_q = env.now
    scanner = choose_shortest(scanners)
    with scanner.request() as req2:
        yield req2
        t_start_scan = env.now
        # queue wait at scanner
        wait_scan = t_start_scan - t_enter_scan_q
        yield env.timeout(random.uniform(scan_min, scan_max)) # service

    # record queue-only waits after warmup
    if arrival_time >= WARMUP:
        stats['id'].append(wait_id)
        stats['scan'].append(wait_scan)
        stats['total'].append(wait_id + wait_scan)

def source(env, id_checkers, scanners, stats):
    i = 0
    while True:
        # Poisson arrivales --> exponential interarrival
        inter = random.expovariate(LAMBDA)
        yield env.timeout(inter)
        i += 1
        env.process(passenger(env, f"P{i}", id_checkers, scanners, stats))

def run_system(num_id_checkers=4, num_scanners = 4, seed=random_seed):
    random.seed(seed)
    env = simpy.Environment()
    # ID/Boardin-Pass check modeled as one block with capacity = number of checkers
    id_checkers = simpy.Resource(env, capacity=num_id_checkers)
    # Several personal-check queues (each capacity = 1)
    scanners = [simpy.Resource(env, capacity = 1) for i in range(num_scanners)]

    stats = {'id': [], 'scan': [], 'total': []}
    env.process(source(env, id_checkers, scanners, stats))
    env.run(until=sim_time)

    def avg(x): return sum(x)/len(x) if x else 0.0
    return {
        'avg_wait_id_min': avg(stats['id']),
        'avg_wait_scan_min': avg(stats['scan']),
        'avg_wait_total_min': avg(stats['total']),
        'n_obs': len(stats['total'])
    }

rows = []
for idc in [3,4,5,6]:
    for scn in [3,4,5,6,8]:
        # do a few reps for stability
        reps = [ run_system(idc, scn, seed=random_seed + r*97 + idc*11 + scn*5) for r in range(5)]
        avg_id = statistics.mean(r['avg_wait_id_min'] for r in reps)
        avg_scan = statistics.mean(r['avg_wait_scan_min'] for r in reps)
        avg_tot = statistics.mean(r['avg_wait_total_min'] for r in reps)
        rows.append({'ID_Checkers': idc, 'Scanners': scn,
                     'AvgWait_ID': avg_id, 'AvgWait_Scan': avg_scan,
                     'AvgWait_Total': avg_tot, 
                     'Meets(<15)': avg_tot < 15})
df = pd.DataFrame(rows).sort_values(['Meets(<15)', 'AvgWait_Total', 'ID_Checkers', 'Scanners'],
                                    ascending = [False, True, True, True])

df


,ID_Checkers,Scanners,AvgWait_ID,AvgWait_Scan,AvgWait_Total,Meets(<15)
19,6,8,0.074736,0.429419,0.504154,True
18,6,6,0.075493,0.443748,0.519241,True
17,6,5,0.076043,0.512839,0.588881,True
14,5,8,0.269633,0.429820,0.699452,True
13,5,6,0.287516,0.443154,0.730670,True
12,5,5,0.278026,0.517810,0.795836,True
16,6,4,0.075013,1.657632,1.732645,True
11,5,4,0.284520,1.703092,1.987612,True
8,4,6,2.321482,0.443185,2.764667,True
9,4,8,2.688628,0.430367,3.118994,True
